In [94]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    torch_dtype="auto",
    attn_implementation="flash_attention_2",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", use_fast=True)

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [95]:
import base64
from io import BytesIO
import os
from decord import VideoReader
from PIL import Image
import numpy as np

import sys
sys.path.append("/projects/cft_vlm/finetune")
from qwenvl.common_utils import make_model_input
processor.tokenizer.padding_side = "right"
message = [
  [
      {
          "role": "system",
          "content": "You are a student in medicine studying for a final exam."
      },
      {
          "role": "user",
          "content": [
              {"type": "text", "text": "Describe this video"},
              {"type": "video", "video": "/projects/cft_vlm/openbiomedvid/vid_processed/-0WrCtDe6IQ_8_567.mp4"},
          ]
      }
  ],
  [
      {
          "role": "system",
          "content": "You are a student in medicine studying for a final exam."
      },
      {
          "role": "user",
          "content": [
              {"type": "text", "text": "Describe this video"},
              {"type": "video", "video": "/projects/cft_vlm/openbiomedvid/vid_processed/-0zSOhy4mgo_1220_1265.mp4"}
          ]
      }
  ],  
]

inputs = make_model_input(
    message[0],
    processor,
    base_interval=1,
    video_min_frames=8,
    video_max_frames=16
)

Unused or unrecognized kwargs: return_tensors, fps.


{151648, 151649, 151650, 151651, 151652, 151653, 151654, 151655, 151656, 151643, 151644, 151645, 151646, 151647}


In [97]:
from qwenvl.common_utils import make_model_input

base_interval = 1
video_min_frames = 8
video_max_frames = 16
conversations = message

data_dict = make_model_input(
    conversations,
    processor,
    base_interval=base_interval,
    video_min_frames=video_min_frames,
    video_max_frames=video_max_frames,
    add_generation_prompt=True,
)
# out = model.generate(**data_dict, max_new_tokens=30, do_sample=False)

out_txt = processor.tokenizer.batch_decode(data_dict['input_ids'], skip_special_tokens=False)
out_txt

Unused or unrecognized kwargs: return_tensors, fps.


{151648, 151649, 151650, 151651, 151652, 151653, 151654, 151655, 151656, 151643, 151644, 151645, 151646, 151647}


['<|im_start|>system\nYou are a student in medicine studying for a final exam.<|im_end|>\n<|im_start|>user\nDescribe this video<|vision_start|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|><|video_pad|

In [15]:
data_dict.keys()

dict_keys(['input_ids', 'attention_mask', 'pixel_values_videos', 'video_grid_thw', 'second_per_grid_ts', 'labels'])

In [27]:
out2 = model.generate(**inputs, max_new_tokens=30, do_sample=False)
out2 = processor.tokenizer.batch_decode(out2, skip_special_tokens=True)
out2

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/fs01/projects/cft_vlm/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:2479: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


['system\nYou are a helpful assistant.\nuser\nDescribe this video\nassistant\nThe provided images are ultrasound scans of the liver, showing various normal and abnormal conditions. Here is a detailed description of each image:\n\n1. **Normal']

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/projects/cft_vlm/finetune")

In [5]:
import datasets
import json

ds = list()
with open("/projects/cft_vlm/openbiomedvid/processed_for_qwen_new.jsonl", "r") as f:
  for line in f:
    ds.append(json.loads(line))
  
with open("/projects/cft_vlm/openbiomedvid/processed_for_qwen_new.jsonl", "w") as f:
  for conversation in ds:
    f.write(json.dumps({"conversation": conversation, "token_count": -1}) + "\n")



In [10]:
ds = datasets.Dataset.from_json("/projects/cft_vlm/openbiomedvid/processed_for_qwen_new.jsonl")

with open("/projects/cft_vlm/openbiomedvid/processed_for_qwen_old.jsonl", "w") as f:
  for row in ds:
    d = {"video": None, "conversations": list(), "token_count": -1}
    for message in row['conversation']:
      m = {
          "from": 'human' if message['role'] == 'user' else 'gpt',
          "value": "",
      }
      for content in message['content']:
        if content['type'] == 'video':
          d["video"] = content['video']
          m['value'] += "<video>"
        else:
          m['value'] += content['text']
      d['conversations'].append(m)
    f.write(json.dumps(d) + "\n")

In [12]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", use_fast=True)

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [161]:
from qwenvl.data.data_qwen_packed_new import new_make_supervised_data_module_packed
from qwenvl.data.data_qwen_packed import make_supervised_data_module_packed
from qwenvl.train.argument import DataArguments

args_new = DataArguments(
    dataset_use='openbiomedvid')
args_old = DataArguments(
    dataset_use='openbiomedvid_old')

new_data_module = new_make_supervised_data_module_packed(
  processor=processor,
  data_args=args_new,
)
old_data_module = make_supervised_data_module_packed(
  processor=processor,
  data_args=args_old,
)
new_ds = new_data_module['train_dataset']
new_collate = new_data_module['data_collator']
new_item = new_ds[1]
old_ds = old_data_module['train_dataset']
old_collate = old_data_module['data_collator']
old_item = old_ds[1]


Unused or unrecognized kwargs: return_tensors, fps.
Unused or unrecognized kwargs: return_tensors.


In [162]:
new_input = new_collate([new_item])
old_input = old_collate([old_item])

In [164]:
new_input['attention_mask'].shape

torch.Size([2])

In [ ]:
import torch
input_ids = new_input['input_ids']
tokenizer = processor.tokenizer

ignore_idx = -100
assistant_start = "<|im_start|>assistant\n"
assistant_end = "<|im_end|>"
labels = torch.ones_like(input_ids) * ignore_idx
a_start_ids = tokenizer.encode(assistant_start, return_tensors='pt')[0]
a_end_ids = tokenizer.encode(assistant_end, return_tensors='pt')[0]
assert len(a_end_ids) == 1
a_end_idx = list(zip(*torch.where(input_ids == a_end_ids[0])))
a_start_idx = list(zip(*torch.where(input_ids == a_start_ids[0])))

print(len(a_start_idx), len(a_end_idx))
assert len(a_start_idx) == len(a_end_idx)
for (batch_idx, start_idx), (batch_idx, end_idx) in zip(a_start_idx, a_end_idx):
  assert batch_idx == batch_idx, "Batch indices do not match"
  assert start_idx < end_idx, "Start index must be less than end index"
  start_end_idx = start_idx + len(a_start_ids)
  print(start_idx, end_idx, start_end_idx)
  if torch.equal(input_ids[batch_idx, start_idx:start_end_idx], a_start_ids):
    end_idx += 1 # Adjust end index to include the end token
    labels[batch_idx, start_end_idx + 1:end_idx] = input_ids[batch_idx, start_end_idx + 1:end_idx]
  else:
    print(input_ids[batch_idx, start_idx:start_end_idx], a_start_ids)
    
labels

(tensor([151644,  77091,    198]), tensor([151645]))

In [163]:
new_input['labels']

tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -

In [ ]:
print(special_ids)
for i in range(input_ids.shape[0]):
  for j in range(input_ids.shape[1]):
    if input_ids[i, j].item() in special_ids:
      labels[i, j] = ignore_idx
